# Neural network demo
This notebook demonstrates the use of a neural network built and trained with _[Keras](http://keras.io)_ with the higgsml dataset and some features of the scikit-learn library.

---

## 1. Read data
Import the required data and split into features, class labels, and weights.

In [1]:
import pandas as pd
df = pd.read_csv('data/atlas-higgs-challenge-2014-v2.csv')

In [2]:
classes = (df['Label'] == 's').astype(int)
weights = df['Weight']
data = df.drop(['EventId', 'Weight', 'Label', 'KaggleSet', 'KaggleWeight'], axis=1)

## 2. Prepare data
Split into 50% train/test samples.

In [3]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(data, classes, weights, test_size=0.5)

Reweight the training samples to equal importance for signal and background.

In [4]:
reweight = lambda w: w / (2 * w.sum())
w_train_normed = w_train.groupby(y_train).transform(reweight) * w_train.shape[0]

Scale the features to remove the mean and have unit variance.

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 3. Define and train neural network
For a simple single-layer perceptron, use the `Sequential` model from Keras with one layer after the input and a single-node output. Note that the first call to `model.add` defines both the input and hidden layers. The hidden layer has 10 units.

In [6]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5005)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 10)            310         dense_input_1[0][0]              
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 1)             11          dense_1[0][0]                    
Total params: 321
____________________________________________________________________________________________________


Use the binary cross-entropy as the loss function, and the Adam optimisation algorithm with default configuration. In the training step, tell Keras to use 20% of the data for testing between epochs. Note that this means we are using a nested cross-validation method.

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train, y_train,
          batch_size=32,
          nb_epoch=10,
          verbose=1,
          validation_split=.2,
          sample_weight=w_train_normed.values)

Train on 327295 samples, validate on 81824 samples
Epoch 1/10
327295/327295 [==============================] - 5s - loss: 0.4202 - val_loss: 0.3838
Epoch 2/10
327295/327295 [==============================] - 5s - loss: 0.3793 - val_loss: 0.3665
Epoch 3/10
327295/327295 [==============================] - 5s - loss: 0.3652 - val_loss: 0.3544
Epoch 4/10
327295/327295 [==============================] - 5s - loss: 0.3565 - val_loss: 0.3485
Epoch 5/10
327295/327295 [==============================] - 5s - loss: 0.3516 - val_loss: 0.3442
Epoch 6/10
327295/327295 [==============================] - 5s - loss: 0.3488 - val_loss: 0.3443
Epoch 7/10
327295/327295 [==============================] - 5s - loss: 0.3470 - val_loss: 0.3408
Epoch 8/10
327295/327295 [==============================] - 5s - loss: 0.3458 - val_loss: 0.3403
Epoch 9/10
327295/327295 [==============================] - 5s - loss: 0.3447 - val_loss: 0.3407
Epoch 10/10
327295/327295 [==============================] - 5s - loss: 0.34

## 4. Evaluate trained model
The Keras model can bne used with scikit-learn functions to evaluate its performance.

In [9]:
from sklearn.metrics import roc_auc_score
y_hat = model.predict(X_test)
roc_auc_score(y_test, y_hat)

0.87565334826750862